In [59]:
!pip install -q pyinterval
from interval import interval
from interval import imath
import pandas as pd
import numpy as np
import math
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_midpoint(interv):
    return (interv[0].sup + interv[0].inf) / 2

def interval_to_str(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

In [60]:
def krawczyk(f, df, start, end, e, tbl):
    x = interval([start, end])
    width = calculate_width(x)
    Fx = f(x)

    # If no 0 in F(X) -> no root
    if 0 not in Fx:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
        return

    # If interval width less than epsilon -> root
    if width < e:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, 'Root']
        return

    # Split interval if 0 in F'(X)
    x_middle = calculate_midpoint(x)
    dFx = df(x)
    if 0 in dFx:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
        krawczyk(f, df, start, x_middle, e, tbl)
        krawczyk(f, df, x_middle, end, e, tbl)
        return

    # Stop if x_i+1 is empty
    dFx_middle = df(x_middle)
    K = x_middle - f(x_middle) / dFx_middle + (1 - dFx / dFx_middle)*(x - x_middle)
    x_next = K & x
    if not x_next:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
        return

    # Display interval if shrunk or not
    tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
    krawczyk(f, df, x_next[0].inf, x_next[0].sup, e, tbl)

In [61]:
func = lambda x: imath.sin(x-1)*(x-2)*(x-3)
dfunc = lambda x: (x**2-5*x+6)*imath.cos(x-1)-(5-2*x)*imath.sin(x-1)
e = 1e-6
a = 0.9
b = 2.3

In [62]:
krawczyk_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Root'])
krawczyk(func, dfunc, a, b, e, krawczyk_result)
krawczyk_result

,Interval,Interval extension,Width,Root
0,"[0.9000000, 2.3000000]","[-0.6070417, 2.2258194]",1.400000e+00,
1,"[0.9000000, 1.6000000]","[-0.2306152, 1.3043241]",7.000000e-01,
2,"[0.9000000, 1.2500000]","[-0.2306152, 0.5715031]",3.500000e-01,
3,"[0.9000000, 1.0750000]","[-0.2306152, 0.1730876]",1.750000e-01,
4,"[0.9618359, 1.0377099]","[-0.0807338, 0.0797734]",7.587406e-02,
5,"[0.9927271, 1.0072727]","[-0.0147047, 0.0147044]",1.454565e-02,
6,"[0.9997350, 1.0002650]","[-0.0005303, 0.0005303]",5.300920e-04,
7,"[0.9999996, 1.0000004]","[-0.0000007, 0.0000007]",7.025497e-07,Root
8,"[1.0750000, 1.2500000]","[0.0983452, 0.4405337]",1.750000e-01,
9,"[1.2500000, 1.6000000]","[0.1385462, 0.7410932]",3.500000e-01,
